In [21]:
import copy
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from typing import Union, List, Dict, Callable, Optional

In [22]:
def calc_series_stats(series: Union[List[float], np.array],
                      stats: Dict[str, Callable]={'mean': np.mean,
                                                  'median': np.median,
                                                  'max': np.max,
                                                  'min': np.min,
                                                  'std': np.std},
                      name_prefix: str='',
                      norm: bool=False) -> Dict[str, float]:
    '''
    Calculate base statistics on series
            
    Parameters
    ----------
    series:
        series by which statistics are calculated
    name_prefix:
        string prefix of returned features
    norm:
        normilize resulted statistics to first element or not
        
    Returns
    -------
        Dict with calculated features 
    '''
    series = np.array(series).astype('float')
    series = series[~np.isnan(series)] 
    series = list(series)
    if len(series) == 0:
        series = np.array([np.nan])
        
    result = {'{}_{}'.format(name_prefix, key): stats[key](series) 
              for key in stats}
    
    if norm:
        result = {key: result[key] / np.abs(series[0]) for key in result}
    
    return result

In [23]:
class QuarterlyFeatures:
    '''
    Feature calculator for qaurtrly-based statistics. 
    Return features for company quarter slices.
    '''
    def __init__(self,
                 data_key: str,
                 columns: List[str],
                 quarter_counts: List[int]=[2, 4, 10],
                 max_back_quarter: int=10,
                 min_back_quarter: int=0,
                 stats: Dict[str, Callable]={'mean': np.mean,
                                             'median': np.median,
                                             'max': np.max,
                                             'min': np.min,
                                             'std': np.std},
                 calc_stats_on_diffs: bool=True,
                 data_preprocessing: Callable=None,
                 n_jobs: int=cpu_count(),
                 verbose: bool=False):
        '''     
        Parameters
        ----------
        data_key:
            key of dataloader in ``data`` argument during 
            :func:`~ml_investment.features.QuarterlyFeatures.calculate`
        columns:
            column names for feature calculation(like revenue, debt etc)
        quarter_counts:
            list of number of quarters for statistics calculation. 
            e.g. if ``quarter_counts = [2]`` than statistics will be calculated
            on current and previous quarter
        max_back_quarter:
            max bound of company slices in time. 
            If ``max_back_quarter = 1`` than features will be calculated
            for only current company quarter. 
            If max_back_quarter is larger than total number of
            quarters for company than features will be calculated 
            for all quarters 
        min_back_quarter:
            min bound of company slices in time. 
            If ``min_back_quarter = 0`` (default) than features will be calculated
            for all quarters. 
            If ``min_back_quarter = 2`` than current and previous quarter slices 
            will not be used for feature calculation 
        stats:
            aggregation functions for features calculation.
            Should be as ``Dict[str, Callable]``.
            Keys of this dict will be used as features names prefixes.
            Values of this dict should implement 
            ``foo(x:List) -> float`` interface
        calc_stats_on_diffs:
            calculate statistics on series diffs( ``np.diff(series)`` ) or not
        data_preprocessing:
            function implemening ``foo(x) -> x_`` interface. 
            It will be used before feature calculation.
        n_jobs:
            number of threads for calculation
        verbose:
            show progress or not
        '''
        self.data_key = data_key
        self.columns = columns
        self.quarter_counts = quarter_counts
        self.max_back_quarter = max_back_quarter
        self.min_back_quarter = min_back_quarter
        self.stats = stats
        self.calc_stats_on_diffs = calc_stats_on_diffs
        self.data_preprocessing = data_preprocessing
        self.n_jobs = n_jobs
        self.verbose = verbose
        self._data_loader = None
        

    def _calc_series_feats(self, data: pd.DataFrame,
                           str_prefix: str='') -> Dict[str, float]:
        result = {}
        for quarter_cnt in self.quarter_counts:
            for col in self.columns:
                series = data[col].values[:quarter_cnt][::-1].astype('float')
                name_prefix = 'quarter{}_{}'.format(quarter_cnt, col)

                feats = calc_series_stats(series=series,
                                          stats=self.stats,
                                          name_prefix=name_prefix)
                result.update(feats)

                if self.calc_stats_on_diffs:
                    diff_feats = calc_series_stats(series=np.diff(series),
                                                   stats=self.stats,
                                                   name_prefix='{}_diff'\
                                                    .format(name_prefix))
                    result.update(diff_feats)
                                
        return result  
        
        
    def _single_ticker(self, ticker:str) -> List[Dict[str, float]]:
        result = []
        #quarterly_data = self._data_loader.load([ticker])
        quarterly_data = pd.read_excel('report data/{}.xlsx'.format(ticker))
        
        if quarterly_data is None:
            return result

        if self.data_preprocessing is not None:
            quarterly_data = self.data_preprocessing(quarterly_data)
        
        max_back_quarter = min(self.max_back_quarter, len(quarterly_data) - 1)
        min_back_quarter = min(self.min_back_quarter, len(quarterly_data) - 1)
        assert min_back_quarter <= max_back_quarter
        
        for back_quarter in range(min_back_quarter, max_back_quarter):
            curr_data = quarterly_data[back_quarter:]

            feats = {
                'ticker': ticker, 
                'date': curr_data['Date'].values[0],
            }

            series_feats = self._calc_series_feats(curr_data)
            feats.update(series_feats)
            
            result.append(feats)
           
        return result
        
        
    def calculate(self, data: pd.DataFrame, index: List[str]) -> pd.DataFrame:
        '''     
        Interface to calculate features for tickers 
        based on data
        
        Parameters
        ----------
        data:
            dict having field named as value in ``data_key`` param of 
            :func:`~ml_investment.features.QuarterlyFeatures.__init__`
            This field should contain class implementing
            ``load(index) -> pd.DataFrame`` interface
        index:
            list of tickers to calculate features for, i.e. ``['AAPL', 'TSLA']``
                      
        Returns
        -------
        ``pd.DataFrame``
            resulted features with index ``['ticker', 'date']``.
            Each row contains features for ``ticker`` company 
            at ``date`` quarter
        '''
        if self.verbose:
            print("Quarterly features calculation")

        #self._data_loader = data[self.data_key]
        X = []
        self._data_loader = data
        for ticker_feats_arr in (self._single_ticker(index)):    
            X.extend(ticker_feats_arr)

        X = pd.DataFrame(X).set_index(['ticker', 'date'])
        
        return X

In [24]:
fc1 = QuarterlyFeatures(data_key='quarterly',
                            columns=['Revenue','Net Debt'],
                            quarter_counts=[2,4,10],
                            max_back_quarter=4,
                            min_back_quarter=0,
                            verbose=True)

In [25]:
data = pd.read_excel('report data/A.xlsx')
data = data.drop('Unnamed: 0', 1)
data.head()

,Date,Revenue,+ Sales & Services Revenue,- Cost of Revenue,+ Cost of Goods & Services,Gross Profit,+ Other Operating Income,- Operating Expenses,"+ Selling, General & Admin",+ Research & Development,...,Return on Invested Capital,Gross Margin,EBITDA Margin,Operating Margin,Incremental Operating Margin,Pretax Margin,Income before XO Margin,Net Income Margin,Net Income to Common Margin,Effective Tax Rate
0,2021-09-30,1586,1586,730,730,856,0,427.151899,388,109,...,0.151456,0.539723,0.323359,0.270396,0.541071,0.259677,0.217190,0.217190,0.217190,0.163617
1,2021-06-30,1525,1525,705,705,820,0,506.000000,398,107,...,0.135745,0.537705,0.256393,0.205902,0.414634,0.196721,0.170164,0.170164,0.170164,0.135000
2,2021-03-31,1548,1548,708,708,840,0,488.000000,385,103,...,0.124215,0.542636,0.276486,0.227390,0.549738,0.220284,0.174354,0.174354,0.174354,0.208503
3,2020-12-31,1483,1483,695,695,788,0,463.000000,387,102,...,0.113722,0.531355,0.278490,0.219150,0.344828,0.208361,0.169361,0.169361,0.169361,0.187177
4,2020-09-30,1261,1261,591,591,670,0,417.000000,328,90,...,0.115248,0.531324,0.269627,0.200634,-0.923077,0.191911,0.169563,0.169563,0.169563,0.116450


In [19]:
fc1.calculate(data, 'A')

Quarterly features calculation


KeyError: "None of ['ticker', 'date'] are in the columns"

In [48]:
import json

In [49]:
def load_json(path):
    with open(path, "r") as read_file:
        in_data = json.load(read_file)
        
    return in_data

In [50]:
def _load_df(json_path: str) -> pd.DataFrame:
    '''
    Load data from SF1 .json file and convert it to pd.DataFrame
    
    Parameters
    ----------
    json_path:
        path to SF1 .json path
        
    Returns
    -------
        ``pd.DataFrame`` content of file
    '''
    data = load_json(json_path)
    df = pd.DataFrame(data['datatable']['data'])
    if len(df) == 0:
        columns = [x['name'] for x in data['datatable']['columns']]
        df = pd.DataFrame(columns=columns, dtype=object)
    else:
        df.columns = [x['name'] for x in data['datatable']['columns']]

    df = df.infer_objects()
    
    return df

In [51]:
class SF1QuarterlyData: 
    '''
    Loader for quartely fundamental information about
    companies(debt, revenue etc)
    '''
    def __init__(self,
                 data_path: Optional[str]=None,
                 quarter_count: Optional[int]=None,
                 dimension: Optional[str]='ARQ'):
        '''
        Parameters
        ----------
        data_path:
            path to :mod:`~ml_investment.data_loaders.sf1` dataset folder
            If None, than will be used ``sf1_data_path``
            from `~/.ml_investment/config.json`
        quarter_count:
            maximum number of last quarters to return. 
            Resulted number may be less due to short history in some companies
        dimension: 
            one of ``['MRY', 'MRT', 'MRQ', 'ARY', 'ART', 'ARQ']``.
            SF1 dataset-based parameter
        '''
        if data_path is None:
            data_path = load_config()['sf1_data_path']
        self.data_path = data_path
        self.quarter_count = quarter_count
        self.dimension = dimension


    def load(self, index: List[str]) -> pd.DataFrame:
        '''    
        Parameters
        ----------
        index:
            list of tickers to load data for, i.e. ``['AAPL', 'TSLA']``
           
        Returns
        -------
        ``pd.DataFrame``
            quarterly information about companies
        '''
        result = []
        for ticker in index:
            path = '{}/{}.json'.format(self.data_path, ticker)
            print(path)
            if not os.path.exists(path):
                continue
            df = _load_df(path)
            df = df[df['dimension'] == self.dimension]
            if self.quarter_count is not None:
                df = df[:self.quarter_count]

            df['date'] = df['datekey']
            df = df.sort_values('date', ascending=False)
            #df = translate_currency(df)
            result.append(df)
        
        if len(result) == 0:
            return None

        result = pd.concat(result, axis=0).reset_index(drop=True)
        result = result.infer_objects()
        result['date'] = result['date'].astype(np.datetime64) 
     
        return result


    def existing_index(self):
        '''  
        Returns
        -------
        ``List``
            existing index values that can pe pushed to `load`
        '''
        dir_path = '{}/core_fundamental'.format(self.data_path)
        index = [x.split('.json')[0] for x in os.listdir(dir_path)]
        return index 

In [52]:
import os
data_loader = SF1QuarterlyData('core_fundamental')

In [53]:
result = data_loader.load(['A'])

core_fundamental/A.json


In [56]:
result

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,date
0,A,ARQ,2020-12-31,2021-03-02,2021-01-31,2021-03-02,-469000000,9674000000,NaN,3483000000,...,306000000,309000000,5.059,5269000000,0,24000000,0,17.219,1796000000,2021-03-02
1,A,ARQ,2020-09-30,2020-12-18,2020-10-31,2021-03-02,-522000000,9627000000,NaN,3415000000,...,309000000,312000000,4.799,5194000000,0,61000000,0,16.809,1948000000,2020-12-18
2,A,ARQ,2020-06-30,2020-09-01,2020-07-31,2021-03-02,-476000000,9546000000,NaN,3245000000,...,309000000,312000000,4.081,5064000000,0,20000000,0,16.388,1931000000,2020-09-01
3,A,ARQ,2020-03-31,2020-06-01,2020-04-30,2021-03-02,-538000000,9455000000,NaN,3171000000,...,309000000,312000000,4.006,4952000000,0,20000000,0,16.026,1226000000,2020-06-01
4,A,ARQ,2019-12-31,2020-03-03,2020-01-31,2021-03-02,-521000000,9501000000,NaN,3102000000,...,310000000,313000000,4.377,4853000000,0,22000000,0,15.655,1210000000,2020-03-03
5,A,ARQ,2019-09-30,2019-12-19,2019-10-31,2021-03-02,-514000000,9452000000,NaN,3189000000,...,308000000,312000000,4.438,4752000000,0,37000000,0,15.429,1109000000,2019-12-19
6,A,ARQ,2019-06-30,2019-08-30,2019-07-31,2021-03-02,-404000000,8625000000,NaN,3457000000,...,312000000,316000000,4.083,5007000000,0,31000000,0,16.048,1835000000,2019-08-30
7,A,ARQ,2019-03-31,2019-05-30,2019-04-30,2021-03-02,-398000000,9022000000,NaN,3812000000,...,317000000,321000000,3.905,5372000000,0,36000000,0,16.946,2694000000,2019-05-30
8,A,ARQ,2018-12-31,2019-03-05,2019-01-31,2021-03-02,-381000000,8952000000,NaN,3712000000,...,318000000,322000000,4.038,5253000000,0,-256000000,0,16.519,2617000000,2019-03-05
9,A,ARQ,2018-09-30,2018-12-20,2018-10-31,2021-03-02,-408000000,8541000000,NaN,3848000000,...,318000000,322000000,4.069,5077000000,0,49000000,0,15.965,2677000000,2018-12-20
